In [1]:
import copy

from itertools import combinations

In [2]:
nodes = ["A", "B", "C"]
node_positions = {v: i for i, v in enumerate(nodes)}
graph = {
    "A": {"B", "C"},
    "B": {"A"},
    "C": {"A"}
}
print("Graph:")
graph

Graph:


{'A': {'B', 'C'}, 'B': {'A'}, 'C': {'A'}}

In [3]:
degree_of_nodes = {n: len(graph[n]) for n in nodes}

print("Degree of all nodes (starting from 0):")
degree_of_nodes # start from 0

Degree of all nodes (starting from 0):


{'A': 2, 'B': 1, 'C': 1}

In [4]:
configurations = {
    tuple([0 for i in range(len(nodes))])
}
# perturb each state at a time for all states in configurations and accumulate the same in the configurations for next state to perturb
for n in nodes:
    node_pos = node_positions[n]
    config_copy = copy.deepcopy(configurations)
    for i in range(1, degree_of_nodes[n]+1):
        for cc in config_copy:
            cc = list(cc)
            cc[node_pos] = i
            configurations.add(tuple(cc))
            
print("All possible configurations:")
configurations, len(configurations)

All possible configurations:


({(0, 0, 0),
  (0, 0, 1),
  (0, 1, 0),
  (0, 1, 1),
  (1, 0, 0),
  (1, 0, 1),
  (1, 1, 0),
  (1, 1, 1),
  (2, 0, 0),
  (2, 0, 1),
  (2, 1, 0),
  (2, 1, 1)},
 12)

In [5]:
invariants = set()
for state in configurations:
    all_paths = combinations(range(len(state)), 2)
    for path in all_paths:
        source = nodes[path[0]]
        dest = nodes[path[1]]
        source_color = state[path[0]]
        dest_color = state[path[1]]
        if dest in graph[source] and source_color == dest_color:
            # found same color node between neighbors
            break
    else:
        invariants.add(state)

print("Invariants and Count of Invariants:")
invariants, len(invariants)

Invariants and Count of Invariants:


({(0, 1, 1), (1, 0, 0), (2, 0, 0), (2, 0, 1), (2, 1, 0), (2, 1, 1)}, 6)

In [6]:
path_information_avg_rank = {}
for inv in invariants:
    path_information_avg_rank[inv] = {"totalPathLength": 0, "pathCount": 1}

print("Path information (Avg rank) for invariants:")
path_information_avg_rank

Path information (Avg rank) for invariants:


{(2, 1, 1): {'totalPathLength': 0, 'pathCount': 1},
 (2, 1, 0): {'totalPathLength': 0, 'pathCount': 1},
 (2, 0, 1): {'totalPathLength': 0, 'pathCount': 1},
 (1, 0, 0): {'totalPathLength': 0, 'pathCount': 1},
 (2, 0, 0): {'totalPathLength': 0, 'pathCount': 1},
 (0, 1, 1): {'totalPathLength': 0, 'pathCount': 1}}

In [7]:
def get_program_transitions_state(start_state):
    result = set()
    for position, val in enumerate(start_state):
        possible_node_colors = set(range(degree_of_nodes[nodes[position]]+1))
        for perturb_val in possible_node_colors:
            perturb_state = list(start_state)
            perturb_state[position] = perturb_val
            perturb_state = tuple(perturb_state)
            if perturb_state != start_state:
                result.add(perturb_state)
    return result

In [8]:
program_transitions = {}

for state in configurations:
    program_transitions[state] = get_program_transitions_state(state)

print("All Program transitions:")
program_transitions, len(program_transitions)

All Program transitions:


({(2, 1, 1): {(0, 1, 1), (1, 1, 1), (2, 0, 1), (2, 1, 0)},
  (1, 0, 1): {(0, 0, 1), (1, 0, 0), (1, 1, 1), (2, 0, 1)},
  (1, 1, 0): {(0, 1, 0), (1, 0, 0), (1, 1, 1), (2, 1, 0)},
  (0, 1, 0): {(0, 0, 0), (0, 1, 1), (1, 1, 0), (2, 1, 0)},
  (2, 1, 0): {(0, 1, 0), (1, 1, 0), (2, 0, 0), (2, 1, 1)},
  (0, 0, 0): {(0, 0, 1), (0, 1, 0), (1, 0, 0), (2, 0, 0)},
  (1, 0, 0): {(0, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0)},
  (2, 0, 1): {(0, 0, 1), (1, 0, 1), (2, 0, 0), (2, 1, 1)},
  (2, 0, 0): {(0, 0, 0), (1, 0, 0), (2, 0, 1), (2, 1, 0)},
  (0, 0, 1): {(0, 0, 0), (0, 1, 1), (1, 0, 1), (2, 0, 1)},
  (1, 1, 1): {(0, 1, 1), (1, 0, 1), (1, 1, 0), (2, 1, 1)},
  (0, 1, 1): {(0, 0, 1), (0, 1, 0), (1, 1, 1), (2, 1, 1)}},
 12)

In [9]:
unranked_states = set(program_transitions.keys()) - set(path_information_avg_rank.keys())
print("Unranked states:")
unranked_states, len(unranked_states)

Unranked states:


({(0, 0, 0), (0, 0, 1), (0, 1, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)}, 6)

In [ ]:
# rank the states that has all the paths to the ranked one
while unranked_states:
    ranked_states = set(path_information_avg_rank.keys())
    remove_from_unranked_states = set()
    for state in unranked_states:
        destinations = program_transitions[state]
        print(state, " destinations", destinations)
        if destinations - ranked_states:
            pass
        else:
            # all the destination has been ranked
            total_path_length = len(destinations)
            path_count = 0
            for succ in destinations:
                path_count += path_information_avg_rank[succ]["pathCount"]
                total_path_length += path_information_avg_rank[succ]["totalPathLength"]
            path_information_avg_rank[state] = {"totalPathLength": total_path_length, "pathCount": path_count}
            remove_from_unranked_states.add(state)
    print("remove ", remove_from_unranked_states)
    unranked_states -= remove_from_unranked_states
    input()

(1, 0, 1)  destinations {(2, 0, 1), (1, 0, 0), (0, 0, 1), (1, 1, 1)}
(1, 1, 0)  destinations {(2, 1, 0), (1, 0, 0), (1, 1, 1), (0, 1, 0)}
(0, 1, 0)  destinations {(2, 1, 0), (0, 0, 0), (1, 1, 0), (0, 1, 1)}
(0, 0, 0)  destinations {(1, 0, 0), (0, 0, 1), (0, 1, 0), (2, 0, 0)}
(0, 0, 1)  destinations {(2, 0, 1), (1, 0, 1), (0, 0, 0), (0, 1, 1)}
(1, 1, 1)  destinations {(2, 1, 1), (1, 0, 1), (1, 1, 0), (0, 1, 1)}
remove  set()


In [ ]:
path_information_avg_rank